In [17]:
import pandas as pd
import numpy as np
import operator
import pprint
import multiprocessing as mp
tree={}

In [2]:
def entropy(class_distribution):
    return np.sum(np.log(1/class_distribution) * class_distribution)

In [3]:
def information_gain(entropy_levelbefore, entropy_oflevel, data_points_in_each_node):
    return entropy_levelbefore - (np.sum((entropy_oflevel*data_points_in_each_node)/np.sum(data_points_in_each_node)))

In [4]:
def find_subtree(dataset,feature,side):
    if side == 'left':
        f= feature.split(" ")
        l= dataset[dataset[f[0]] >= float(f[1])]
        return l
    else:
        f= feature.split(" ")
        l= dataset[dataset[f[0]] < float(f[1])]
        return l

In [26]:
def info_for_feature(feature,dataset,prev_entropy,entropy_cache):
    en={}
    info={}
    print(feature)
    for s in set(list(dataset[feature])):
                left= dataset[dataset[feature] >= s]
                right=dataset[dataset[feature] < s]
                
                left_distribution =(np.asarray(list(left['class'].value_counts())))/len(left['class'])
                right_distribution =(np.asarray(list(right['class'].value_counts())))/len(right['class'])
                
                if str(left_distribution) not in entropy_cache:
                    en[feature+" left"] = entropy(left_distribution)
                    entropy_cache[str(left_distribution)]=en[feature+" left"]
                else:
                    en[feature+" left"] =entropy_cache[str(left_distribution)]

                if str(right_distribution) not in entropy_cache:
                    en[feature+" right"] = entropy(right_distribution)
                    entropy_cache[str(right_distribution)]=en[feature+" right"]
                else:
                    en[feature+" right"]=entropy_cache[str(right_distribution)]
               
                info[str(feature)+" "+str(s)]=information_gain(prev_entropy,np.asarray([float(en[feature+" left"]),float(en[feature+" right"])]),np.asarray([len(left),len(right)]))
    return info

In [47]:
def select_feature(dataset,prev_entropy,entropy_cache):
    en={}
    info={}
    cols=dataset.columns
    cols=list(cols)
    del cols[-1]
    p=mp.Pool(3)
    tuple_list=[]
    for c in cols:
        tup= (c,dataset,prev_entropy,entropy_cache)
        tuple_list.append(tup)
    list_dict=p.starmap(info_for_feature,tuple_list)
    for i in list_dict:
        info.update(i)
    best_feature=max(info.items(), key=operator.itemgetter(1))[0]
    return best_feature

In [40]:
def training(dataset,depth,tree,entropy_cache={}):
    current_node_entropy = entropy((np.asarray(list(dataset['class'].value_counts())))/len(dataset['class']))
    if (current_node_entropy>=0.25 and depth !=0):
        
            current_node_entropy = entropy((np.asarray(list(dataset['class'].value_counts())))/len(dataset['class']))
            best_feature = select_feature(dataset,current_node_entropy,entropy_cache)
            print(best_feature)
            left_subtree= find_subtree(dataset,best_feature,"left")
            right_subtree=find_subtree(dataset,best_feature,"right")
            if not left_subtree.empty:
                tree["> "+str(best_feature)]=training(left_subtree,depth-1,{},entropy_cache)
            if not right_subtree.empty:
                tree["< "+str(best_feature)]=training(right_subtree,depth-1,{},entropy_cache)
            return tree
    else:
        return (dataset['class'].value_counts())

In [41]:
data = open('spambase.data').read().split("\n")
data = [i.split(",") for i in (data)]
column = open('features','r').read().split("\n")
column =[i.strip() for i in column]
dataset =  pd.DataFrame(data, columns=column)
dataset=dataset.convert_objects(convert_numeric=True)
dataset=dataset.dropna()
columnmax = [max(dataset[i]) for i in dataset.columns]
columnmin = [min(dataset[i]) for i in dataset.columns]
for i,j in enumerate(dataset.columns):
    dataset[j] = (dataset[j] - columnmin[i])/columnmax[i]
dataset['class']= dataset['class'].astype(int)
# training_dataset=dataset.sample(frac=1)[:-1000]
# test_dataset=dataset.sample(frac=1)[-1000:]

/Users/shaivalpatel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [42]:
def testing(row,tree,depth):
 
    if  depth !=0 and type(tree)==dict :
        for key in tree.keys():
            operator = key.split(" ")[0]
            feature =key.split(" ")[1]
            threshold=key.split(" ")[2]
            row_feature = row[feature]
            if operator == '<':
                if (row_feature)< float(threshold):
                    result=testing(row,tree[key],depth-1)
                    return result
            else:
                if (row_feature)>= float(threshold):
                    result=testing(row,tree[key],depth-1)
                    return result
    else:
        
        if 0 not in tree:
            return 1
        elif 1 not in tree:
            return 0
        else:
            if tree[0]>tree[1]:
                return 0
            else:
                return 1

In [43]:
prediction=[]
for index,row in test_dataset.iterrows():
    prediction.append(testing(row,tree,10))

In [44]:
def accuracy(prediction,test_dataset):
    test=list(test_dataset['class'])
    ac = zip(prediction,test)
    right =0
    for k,v in ac:
        if k ==v:
            right+=1
    return right/len(prediction)

In [45]:
acc = accuracy(prediction,test_dataset)

In [50]:
%%time
ac=[]
dataset = dataset.sample(frac=1)
for i in range(0,len(dataset),920):
    print(i)
    if i+920<len(dataset):
        tree={}
        
        training_dataset=pd.concat([dataset[i+920:],dataset[0:i]])
        test_dataset=dataset[i:i+920]
        tree=training(training_dataset,12,tree)
        prediction=[]
        for index,row in test_dataset.iterrows():
            prediction.append(testing(row,tree,12))
        acc = accuracy(prediction,test_dataset)
        ac.append(acc)
print(np.sum(np.asarray(ac))/5)

0
char_freq_! 0.0024632058624299525
char_freq_$ 0.0021655838747293025
word_freq_remove 0.016506189821182946
capital_run_length_average 0.0015918367346938775
char_freq_! 0.011423117187018904
word_freq_re 0.05602240896358543
word_freq_all 0.30000000000000004
word_freq_will 0.2647362978283351
word_freq_your 0.09900990099009903
word_freq_meeting 0.11344537815126052
word_freq_email 0.06820682068206821
capital_run_length_average 0.003061224489795918
word_freq_free 0.057999999999999996
word_freq_hp 0.010081613058089295
char_freq_( 0.01579163248564397
char_freq_[ 0.08086253369272237
word_freq_all 0.23529411764705882
word_freq_address 0.023109243697478993
capital_run_length_average 0.00199546485260771
word_freq_all 0.15098039215686276
word_freq_free 0.020999999999999998
char_freq_! 0.014502124515056346
word_freq_free 0.034
word_freq_mail 0.05775577557755776
word_freq_will 0.05997931747673216
word_freq_free 0.0715
word_freq_george 0.08640864086408641
char_freq_! 0.023523615986206045
capital_run_